# How does SHA-256 work?
### Simple hasing algorithm 256 (256 indicates the length of the digest)

In [2]:
# Needed modules for running all notebook cells
from IPython.display import clear_output
from bitarray import bitarray
from hashlib import sha256
from time import sleep
from math import sqrt, floor

In [3]:
m = sha256()
print(f"Hash type:\t{m.name}")
print(f"Void digest:\t{m.hexdigest()}")
print(f"Digest size:\t{m.digest_size}")

string = "Hello World!"
m.update(string.encode())
print(f"\nString:\t{string}")
print(f"Digest:\t{m.hexdigest()}")

del m, string

Hash type:	sha256
Void digest:	e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
Digest size:	32

String:	Hello World!
Digest:	7f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069


## Atomic bit level operations

1. [SHR](#SHR)
2. [ROTR](#ROTR)
3. [XOR](#XOR)
4. [SUM](#SUM)

<a ID="SHR"></a>
### 1. SHR
Shift right

In [4]:
def shr(bits, shifts):
    for _ in range(shifts):
        bits.pop()
        bits.insert(0, 0)
    return bits

a = bitarray(endian='big')
a.frombytes(b'\xff\x00\xff\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
for i in range(32):
    print("\r{:>8} {:>32}".format("SHR "+str(i+1)+":", shr(a,1).to01()), end="")
    sleep(0.2)
del a, i

      A: 11111111000000001111111100000000
 SHR 32: 00000000000000000000000000000000

<a ID="ROTR"></a>
### 2. ROTR
Rotation right

In [5]:
def rotr(bits, shifts):
    for _ in range(shifts):
        bits.insert(0,bits.pop())
    return bits

a = bitarray(endian='big')
a.frombytes(b'\xff\x00\xff\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
for i in range(32):
    print("\r{:>8} {:>32}".format("ROTR "+str(i+1)+":", rotr(a,1).to01()), end="")
    sleep(0.2)
del a, i

      A: 11111111000000001111111100000000
ROTR 32: 11111111000000001111111100000000

<a ID="XOR"></a>
### 3. XOR
Exclusive OR



| A | B | XOR |
|---|---|-----|
| 0 | 0 | 0   |
| 0 | 1 | 1   |
| 1 | 0 | 1   |
| 1 | 1 | 0   |   



In [6]:
def xor(a, b):
    return a^b

a = bitarray(endian='big')
b = bitarray(endian='big')
a.frombytes(b'\xff\x00\xff\x00')
b.frombytes(b'\x00\xff\xff\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
print("{:>8} {:>32} XOR".format("B:", b.to01()))
print("{:>8} {:>32}".format("", "-"*32))
for i in range(32):
    print("\r{:>8} {:>32}".format("", xor(a,b).to01()[32-1-i:]), end="")
    sleep(0.2)
del a, b, i


      A: 11111111000000001111111100000000
      B: 00000000111111111111111100000000 XOR
         --------------------------------
         11111111111111110000000000000000

<a ID="SUM"></a>
### 4. SUM
Binary addition <br>
Mod 32, ignoring the carry out that will overflow the 32 bit.

In [7]:
def add(a, b):
    result = bitarray(endian='big')
    carry = 0
    for i in range(32-1, -1, -1):
        result.insert(0,(a[i]^b[i]^carry))
        carry = 1 if (a[i] + b[i] + carry) > 1 else 0
    return result


a = bitarray(endian='big')
b = bitarray(endian='big')
a.frombytes(b'\x80\x30\xff\xff')
b.frombytes(b'\x80\x50\x00\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
print("{:>8} {:>32} +".format("B:", b.to01()))
print("{:>8} {:>32}".format("", "-"*32))
out = add(a,b)
for i in range(32):
    print("\r{:>8} {:>32}".format("", out.to01()[32-1-i:]), end="")
    sleep(0.2)
del a, b, i

      A: 10000000001100001111111111111111
      B: 10000000010100000000000000000000 +
         --------------------------------
         00000000100000001111111111111111

***
## Compound operations

1. [$\sigma0$](#sigma0)
2. [$\sigma1$](#sigma1)
3. [$\Sigma0$](#Sigma0)
4. [$\Sigma1$](#Sigma1)
5. [Choice](#Choice)
6. [Majority](#Majority)


<a ID="sigma0"></a>
### 1. $\sigma0$

In [8]:
def sigma0(a, b, c):
    rotr(a, 7)
    rotr(b, 18)
    shr(c, 3)
    return xor(xor(a,b),c)

origin = bitarray(endian='big')
origin.frombytes(b'\x00\x00\x3f\xff')

a = origin.copy()
b = origin.copy()
c = origin.copy()

result = sigma0(a, b, c)

print("{:>8} {:>32}".format("Origin:", origin.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("ROTR  7:", a.to01()))
print("{:>8} {:>32} XOR".format("ROTR 18:", b.to01()))
print("{:>8} {:>32} XOR".format("SHR  7:", c.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("\u03C30:", result.to01()))

del a, b, c, result, origin



 Origin: 00000000000000000011111111111111
         --------------------------------
ROTR  7: 11111110000000000000000001111111
ROTR 18: 00001111111111111100000000000000 XOR
 SHR  7: 00000000000000000000011111111111 XOR
         --------------------------------
     σ0: 11110001111111111100011110000000


<a ID="sigma1"></a>
### 1. $\sigma1$

In [9]:
def sigma1(a, b, c):
    rotr(a, 17)
    rotr(b, 19)
    shr(c, 10)
    return xor(xor(a,b),c)

origin = bitarray(endian='big')
origin.frombytes(b'\x00\x00\x3f\xff')

a = origin.copy()
b = origin.copy()
c = origin.copy()

result = sigma1(a, b, c)

print("{:>8} {:>32}".format("Origin:", origin.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("ROTR 17:", a.to01()))
print("{:>8} {:>32} XOR".format("ROTR 19:", b.to01()))
print("{:>8} {:>32} XOR".format("SHR 10:", c.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("\u03C31:", result.to01()))

del a, b, c, result, origin


 Origin: 00000000000000000011111111111111
         --------------------------------
ROTR 17: 00011111111111111000000000000000
ROTR 19: 00000111111111111110000000000000 XOR
 SHR 10: 00000000000000000000000000001111 XOR
         --------------------------------
     σ1: 00011000000000000110000000001111


<a ID="Sigma0"></a>
### 3. $\Sigma0$

In [10]:
def Sigma0(a, b, c):
    rotr(a, 2)
    rotr(b, 13)
    rotr(c, 22)
    return xor(xor(a,b),c)

origin = bitarray(endian='big')
origin.frombytes(b'\x00\x00\x3f\xff')

a = origin.copy()
b = origin.copy()
c = origin.copy()

result = Sigma0(a, b, c)

print("{:>8} {:>32}".format("Origin:", origin.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("ROTR  2:", a.to01()))
print("{:>8} {:>32} XOR".format("ROTR 13:", b.to01()))
print("{:>8} {:>32} XOR".format("ROTR 22:", c.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("\u03A30:", result.to01()))

del a, b, c, result, origin


 Origin: 00000000000000000011111111111111
         --------------------------------
ROTR  2: 11000000000000000000111111111111
ROTR 13: 11111111111110000000000000000001 XOR
ROTR 22: 00000000111111111111110000000000 XOR
         --------------------------------
     Σ0: 00111111000001111111001111111110


<a ID="Sigma1"></a>
### 4. $\Sigma1$

In [11]:
def Sigma1(a, b, c):
    rotr(a, 6)
    rotr(b, 11)
    rotr(c, 25)
    return xor(xor(a,b),c)

origin = bitarray(endian='big')
origin.frombytes(b'\x00\x00\x3f\xff')

a = origin.copy()
b = origin.copy()
c = origin.copy()

result = Sigma1(a, b, c)

print("{:>8} {:>32}".format("Origin:", origin.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("ROTR  6:", a.to01()))
print("{:>8} {:>32} XOR".format("ROTR 11:", b.to01()))
print("{:>8} {:>32} XOR".format("ROTR 25:", c.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("\u03A31:", result.to01()))

del a, b, c, result, origin

 Origin: 00000000000000000011111111111111
         --------------------------------
ROTR  6: 11111100000000000000000011111111
ROTR 11: 11111111111000000000000000000111 XOR
ROTR 25: 00000000000111111111111110000000 XOR
         --------------------------------
     Σ1: 00000011111111111111111101111000


<a ID="Choice"></a>
### 5. Choice

In [12]:
def choice(choice, a, b):
    result = choice.copy()
    for i in range(32):
        if choice[i] == 1:
            result[i] = a[i]
        elif choice[i] == 0:
            result[i] = b[i]
    return result

a = bitarray(endian='big')
b = bitarray(endian='big')
c = bitarray(endian='big')

a.frombytes(b'\x00\xff\x00\xff')
b.frombytes(b'\x00\x00\xff\xff')
c.frombytes(b'\xff\xff\x00\x00')

result = choice(a, b, c)

for i in range(32):

    print(" "*(41-(i+1))+"\u21D3")
    print("{:>8} {:>32}".format("A:", a.to01()))
    print("{:>8} {:>32} {}".format("B:", b.to01(), "" if a[i]==1 else "\u21E6"))
    print("{:>8} {:>32} {}".format("C:", c.to01(), "" if a[i]==0 else "\u21E6"))
    print("{:>8} {:>32}".format("", "-"*32))
    print("{:>8} {:>32}".format("OUT:", result.to01()[32-(i+1):]))
    
    sleep(0.2)
    clear_output(wait=True)
    

del a, b, c, result

         ⇓
      A: 00000000111111110000000011111111
      B: 00000000000000001111111111111111 
      C: 11111111111111110000000000000000 ⇦
         --------------------------------
    OUT: 11111111000000000000000011111111


<a ID="Majority"></a>
### 6. Majority

In [13]:
def majority(a, b, c):
    result = a.copy()
    for i in range(32):
        result[i] = 1 if a[i]+b[i]+c[i] > 1 else 0
    return result

a = bitarray(endian='big')
b = bitarray(endian='big')
c = bitarray(endian='big')

a.frombytes(b'\x00\xff\x00\xff')
b.frombytes(b'\x00\x00\xff\xff')
c.frombytes(b'\xff\xff\x00\x00')

result = majority(a, b, c)

for i in range(32):

    print(" "*(41-(i+1))+"\u21D3")
    print("{:>8} {:>32} {}".format("A:", a.to01(), " "+str(a[32-(i+1)])))
    print("{:>8} {:>32} {}".format("B:", b.to01(), " "+str(b[32-(i+1)])))
    print("{:>8} {:>32} {}".format("C:", c.to01(), " "+str(c[32-(i+1)])))
    print("{:>8} {:>32}".format("", "-"*32))
    print("{:>8} {:>32}".format("OUT:", result.to01()[32-(i+1):]))
    
    sleep(0.2)
    clear_output(wait=True)
    

del a, b, c, result

         ⇓
      A: 00000000111111110000000011111111  0
      B: 00000000000000001111111111111111  0
      C: 11111111111111110000000000000000  1
         --------------------------------
    OUT: 00000000111111110000000011111111


***
## Constant K
List of length 64 containing 32 bits from the decimal part of the cubic root of the 64 first prime numbers.

In [15]:
first64primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89,
 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197,
 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]

K = [] # The list is indexed from 0 even tho is printed starting form 1.

def frac2bin(frac):
    x = floor(frac*(2**32))
    bin32 = ""
    
    for _ in range(32):
        bin32 += str(x % 2)
        x = x // 2

    return bin32[::-1]
    


for j, i in enumerate(first64primes):
    sqrt3 = i**(1./3.)
    deciaml_part = float(str(sqrt3)[1:])
    K.append(bitarray(frac2bin(deciaml_part)))

    print("{:>4} \u21FE {:>4} \u21FE {:>18} \u21FE {:>18} \u21FE {:>32}  ".format(j+1, "\u221B"+str(i), sqrt3, deciaml_part,K[j].to01()))

del first64primes, deciaml_part, sqrt3, i, j



   1 ⇾   ∛2 ⇾ 1.2599210498948732 ⇾ 0.2599210498948732 ⇾ 01000010100010100010111110011000  
   2 ⇾   ∛3 ⇾ 1.4422495703074083 ⇾ 0.4422495703074083 ⇾ 01110001001101110100010010010001  
   3 ⇾   ∛5 ⇾ 1.7099759466766968 ⇾ 0.7099759466766968 ⇾ 10110101110000001111101111001111  
   4 ⇾   ∛7 ⇾  1.912931182772389 ⇾  0.912931182772389 ⇾ 11101001101101011101101110100101  
   5 ⇾  ∛11 ⇾ 2.2239800905693152 ⇾ 0.2239800905693152 ⇾ 00111001010101101100001001011011  
   6 ⇾  ∛13 ⇾ 2.3513346877207573 ⇾ 0.3513346877207573 ⇾ 01011001111100010001000111110001  
   7 ⇾  ∛17 ⇾  2.571281590658235 ⇾  0.571281590658235 ⇾ 10010010001111111000001010100100  
   8 ⇾  ∛19 ⇾  2.668401648721945 ⇾  0.668401648721945 ⇾ 10101011000111000101111011010101  
   9 ⇾  ∛23 ⇾ 2.8438669798515654 ⇾ 0.8438669798515654 ⇾ 11011000000001111010101010011000  
  10 ⇾  ∛29 ⇾  3.072316825685847 ⇾  0.072316825685847 ⇾ 00010010100000110101101100000001  
  11 ⇾  ∛31 ⇾ 3.1413806523913927 ⇾ 0.1413806523913927 ⇾ 00100100001100011000010110111110  

***
<a ID="Hashing"></a>
## Hashing

1. [Encode](#Encode)
2. [Padding](#Padding)
3. [Block Iterator](#Block-Iterator)



<a ID="Encode"></a>
### Encode
Always use the same encode to have a solid and consistent output; in the case of SHA256 we use ASCII (American Standard Code II)

In [16]:
string = "abc"
message = bitarray(endian='big')
message.frombytes(string.encode('ascii')) # passing it to binary with a ASCII encoding

print("Message:")
print("\t{} \"{}\"".format("Input:", string))
print("\tEncoding: ASCII")
print("\t{} {}".format("Message:", message.to01()))



Message:
	Input: "abc"
	Encoding: ASCII
	Message: 011000010110001001100011


<a ID="Padding"></a>
### Padding
Hashing algorithms work in badges or blocks. Padding adds bits to make the message length a multiple of the block size required.

In [17]:
m = sha256()
print(f"SHA-256 block size: {m.block_size*8} bits")

del m

SHA-256 block size: 512 bits


Pdding structure:
* Padding flag bit, added right after the message.
* Cero padding, ceros added to the message in order to make its length a multiple of the block.
* Length on the message encoded in the last 64 bits.

In [104]:
def padding(message):

    p_message = message.to01()
    blocks = len(message) // 512 +1 # Number of block the message is gonna take

    p_message += '1' # Padding flag

    # Same result as the commented loop
    p_message += '0'*(blocks*512 - 64 - len(p_message))
    #for _ in range(len(p_message), blocks*512 - 64):
    #    p_message += '0'

    # Last 64 bits with message length
    bin_len_message = bin(len(message))[2:]
    len_bin_len_message = len(bin_len_message)
    p_message += '0'*(64-len_bin_len_message) + bin_len_message

    return bitarray(p_message)



p_message = padding(message)

print(f"Padding message size: {len(p_message)} bits")
print("Message:")

colors = {'Message':'\033[95m', 'Padding flag':'\033[91m', 'Padding':'\033[93m', 'Length':'\033[96m', 'Message Length':'\033[94m'}

for i, bit in enumerate(p_message.to01()):
    if i >= len(p_message) - len(bin(len(message))[2:]):
        color = colors['Message Length']
    elif i >= len(p_message) - 64:
        color = colors['Length']
    elif i >= len(message) + 1:
        color = colors['Padding']
    elif i >= len(message):
        color = colors['Padding flag']
    else:
        color = colors['Message']
    print(color+bit, end="")

print(f"\n{colors['Message']}\u2589 {'Message'}")
print(f"{colors['Padding flag']}\u2589 {'Padding flag'}")
print(f"{colors['Padding']}\u2589 {'Padding'}")
print(f"{colors['Length']}\u2589 {'Reserved bits for mesage length'}")
print(f"{colors['Message Length']}\u2589 {'Message length'}")

del color, colors, bit, i

Padding message size: 512 bits
Message:
01100001011000100110001110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011000
▉ Message
▉ Padding flag
▉ Padding
▉ Reserved bits for mesage length
▉ Message length


<a ID="Block-Iterator"></a>
### Block Iterator

In [108]:
def blck_iter(message):
    N_blck = len(message) // 512
    print(N_blck)
    for blck in range(N_blck):
        yield message[blck*512:(blck+1)*512]

In [110]:

for i in blck_iter(p_message):
    print(i)

1
bitarray('01100001011000100110001110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011000')
